# Speech Recognition with Wav2Vec

Initially from article https://pub.towardsai.net/speech-to-text-with-wav2vec-2-0-b21c1e1ad701

Blog on Wav2Vec 2.0: https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/

Initial github drawn from https://github.com/sdhilip200/speech-to-text ut modified as librosa couldnt run on AWS instances as sndfile couldnt install.

Detailed article: https://maelfabien.github.io/machinelearning/wav2vec/#5-the-code



In [1]:
!pip install -r requirements.txt

# HuggingFace Library

In [2]:
# Import necessary library

from audio2numpy import open_audio

#Importing Pytorch
import torch

#Importing Wav2Vec tokenizer
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

In [3]:
# Importing Wav2Vec pretrained model

# tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h") # Deprecated
tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# audio_file = "taken_clip.wav" 
# audio_file = "timeout.wav"
# audio_file = "timeout_out.wav"
# audio_file = "preamble_out.wav"
audio_file = "gettysburg10_out.wav"

In [5]:
# Reading taken audio clip

import IPython.display as display
display.Audio(audio_file, autoplay=True)

In [6]:
audio, rate = open_audio(audio_file)

In [7]:
rate

16000

In [8]:
audio.shape

(160049,)

In [9]:
audio.shape[0] / rate

10.0030625

In [10]:
# audio_arr = audio[:,0]
audio_arr = audio

In [11]:
# Taking an input value

input_values = tokenizer(audio_arr, sampling_rate=rate, return_tensors = "pt").input_values

In [12]:
input_values

tensor([[0.0521, 0.0268, 0.0102,  ..., 0.0146, 0.0460, 0.0547]])

In [13]:
# Storing logits (non-normalized prediction values)
logits = model(input_values).logits

In [14]:
# Storing predicted id's
prediction = torch.argmax(logits, dim = -1)


In [15]:
# Passing the prediction to the tokenzer decode to get the transcription
transcription = tokenizer.batch_decode(prediction)[0]
print(transcription)

FOUR SCORE AND SEVEN YEARS AGO OUR FATHERS BROUGHT FORTH ON THIS CONTINENT A NEW NATION CONCEIVED IN LIBERTY AND DEDICATED TO THE PROPOSITION THAT ALL MEN ARE CREATED EQUAL
